In [49]:
import pymysql
import random
import pandas as pd
import numpy as np
import pyarrow
import csv
import json
import os
import time
import hashlib
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta

# Task 1

In [2]:
HOST = "127.0.0.1"
USER = "TestUser"
PASSWORD = "12341234"
DB_NAME = "Students"
TB_NAME = 'stu'

In [3]:
def connectingToDB():
    try:
        connection = pymysql.connect(
            host=HOST,
            port=3306,
            user=USER,
            password=PASSWORD,
            database=DB_NAME,
            cursorclass=pymysql.cursors.DictCursor
        )
        print("Connection success!")
        return connection
    except Exception as ex:
        print("Connection refused...")
        print(ex)

In [4]:
def closeDB(connection):
    try:    
        connection.close()  
        print("Connection is close!")
    except Exception as ex:
        print("Something went wrong...")
        print(ex)

In [5]:
def insertDB(connection, tabl, namestu, surnamestu, fathmanestu, midmarkstu):
    with connection.cursor() as cursor:
        insert_query = (f"INSERT INTO `{tabl}` (namestu, surnamestu, fathmanestu, midmarkstu) " 
                        f"VALUES ('{namestu}', '{surnamestu}', '{fathmanestu}', '{midmarkstu}');");
        #print(insert_query)
        cursor.execute(insert_query)
        connection.commit()
        

In [6]:
def allDB(connection, tabl):
    with connection.cursor() as cursor:
        cursor.execute(f"SELECT * FROM {tabl}")  
        return list(cursor)

In [7]:
def countDB(connection, tabl):
    with connection.cursor() as cursor:
        cursor.execute(f"SELECT COUNT(*) FROM {tabl}")  
        return list(cursor)

In [8]:
def takeNotAdmittedDB(connection, tabl):
    with connection.cursor() as cursor:
        startSelectTime = time.time()
        cursor.execute(f"SELECT * FROM {tabl} WHERE midmarkstu < 35")  
        print("Time of request: %s" % (time.time() - startSelectTime))
        return list(cursor)

In [9]:
def takeOnTheExamDB(connection, tabl):
    with connection.cursor() as cursor:
        startSelectTime = time.time()
        cursor.execute(f"SELECT * FROM {tabl} WHERE midmarkstu >= 35 AND midmarkstu < 60")  
        print("Time of request: %s" % (time.time() - startSelectTime))
        return list(cursor)

In [10]:
def takeNormalMarkDB(connection, tabl):
    with connection.cursor() as cursor:
        startSelectTime = time.time()
        cursor.execute(f"SELECT * FROM {tabl} WHERE midmarkstu >= 60 AND midmarkstu < 90")  
        print("Time of request: %s" % (time.time() - startSelectTime))
        return list(cursor)

In [11]:
def takeHighMarkDB(connection, tabl):
    with connection.cursor() as cursor:
        startSelectTime = time.time()
        cursor.execute(f"SELECT * FROM {tabl} WHERE midmarkstu >= 90")  
        print("Time of request: %s" % (time.time() - startSelectTime))
        return list(cursor)

In [12]:
names = ['Sergey', 'Maxim', 'Yura', 'Ivan', 'Igor', 'Misha']
surnames = ['Kovalenko', 'Bondar', 'Kravchuk', 'Kyznecov']
fathernames = ['Sergeyevich', 'Maximovich', 'Yurovich', 'Ivanovich', 'Igorevich', 'Mihailovich']

In [13]:
connection = connectingToDB()

Connection success!


In [14]:
#for i in range(100):
#    insertDB(connection, names[random.randint(0, len(names) - 1)], 
#             TB_NAME
#             surnames[random.randint(0, len(surnames) - 1)], 
#             fathernames[random.randint(0, len(fathernames) - 1)], 
#             random.randint(0, 100))    

In [15]:
print(countDB(connection, TB_NAME))

[{'COUNT(*)': 100}]


In [16]:
df = pd.DataFrame(allDB(connection, TB_NAME))
print(df.shape)
df

(100, 5)


,idstu,namestu,surnamestu,fathmanestu,midmarkstu
0,2,Igor,Kovalenko,Sergeyevich,65.0
1,3,Yura,Bondar,Maximovich,60.0
2,4,Yura,Kravchuk,Ivanovich,19.0
3,5,Misha,Kovalenko,Ivanovich,11.0
4,6,Sergey,Kravchuk,Ivanovich,57.0
...,...,...,...,...,...
95,97,Sergey,Bondar,Maximovich,37.0
96,98,Misha,Kyznecov,Ivanovich,81.0
97,99,Yura,Bondar,Igorevich,88.0
98,100,Sergey,Kovalenko,Yurovich,25.0


In [17]:
df = pd.DataFrame(takeNotAdmittedDB(connection, TB_NAME))
print(df.shape)
df

Time of request: 0.000997781753540039
(31, 5)


,idstu,namestu,surnamestu,fathmanestu,midmarkstu
0,4,Yura,Kravchuk,Ivanovich,19.0
1,5,Misha,Kovalenko,Ivanovich,11.0
2,8,Sergey,Kovalenko,Sergeyevich,16.0
3,10,Maxim,Kyznecov,Yurovich,1.0
4,16,Maxim,Kravchuk,Igorevich,31.0
5,17,Ivan,Kovalenko,Ivanovich,16.0
6,21,Sergey,Kyznecov,Mihailovich,10.0
7,22,Yura,Kravchuk,Igorevich,8.0
8,24,Sergey,Kravchuk,Igorevich,26.0
9,25,Maxim,Kovalenko,Igorevich,22.0


In [18]:
df = pd.DataFrame(takeOnTheExamDB(connection, TB_NAME))
print(df.shape)
df

Time of request: 0.000997781753540039
(25, 5)


,idstu,namestu,surnamestu,fathmanestu,midmarkstu
0,6,Sergey,Kravchuk,Ivanovich,57.0
1,7,Yura,Bondar,Ivanovich,41.0
2,9,Igor,Bondar,Ivanovich,45.0
3,13,Yura,Kyznecov,Maximovich,41.0
4,14,Misha,Kovalenko,Igorevich,55.0
5,15,Misha,Kyznecov,Mihailovich,48.0
6,19,Igor,Bondar,Sergeyevich,41.0
7,30,Maxim,Kovalenko,Igorevich,41.0
8,37,Ivan,Kravchuk,Sergeyevich,36.0
9,38,Yura,Kyznecov,Igorevich,49.0


In [19]:
df = pd.DataFrame(takeNormalMarkDB(connection, TB_NAME))
print(df.shape)
df

Time of request: 0.0010008811950683594
(34, 5)


,idstu,namestu,surnamestu,fathmanestu,midmarkstu
0,2,Igor,Kovalenko,Sergeyevich,65.0
1,3,Yura,Bondar,Maximovich,60.0
2,12,Ivan,Kyznecov,Maximovich,68.0
3,18,Igor,Kravchuk,Mihailovich,80.0
4,20,Misha,Kyznecov,Yurovich,87.0
5,23,Yura,Kravchuk,Maximovich,69.0
6,28,Misha,Kyznecov,Sergeyevich,66.0
7,29,Maxim,Kravchuk,Sergeyevich,80.0
8,31,Yura,Kravchuk,Mihailovich,82.0
9,32,Misha,Kravchuk,Mihailovich,79.0


In [20]:
df = pd.DataFrame(takeHighMarkDB(connection, TB_NAME))
print(df.shape)
df

Time of request: 0.0
(10, 5)


,idstu,namestu,surnamestu,fathmanestu,midmarkstu
0,11,Ivan,Bondar,Ivanovich,96.0
1,26,Ivan,Bondar,Igorevich,100.0
2,34,Maxim,Bondar,Yurovich,96.0
3,36,Igor,Kravchuk,Mihailovich,94.0
4,53,Ivan,Kravchuk,Maximovich,94.0
5,71,Maxim,Kravchuk,Sergeyevich,92.0
6,77,Yura,Kovalenko,Sergeyevich,96.0
7,80,Igor,Kyznecov,Yurovich,97.0
8,85,Ivan,Bondar,Yurovich,91.0
9,86,Igor,Kovalenko,Yurovich,91.0


In [21]:
#closeDB(connection)

# Task 2

In [22]:
def md5(path):
    hash_md5 = hashlib.md5()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

In [23]:
def sha256(path):
    hash_sha256 = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_sha256.update(chunk)
    return hash_sha256.hexdigest()

In [24]:
def writeToJson(path, data = {}, rewrite = 0):
    try:
        with open(os.path.join(os.getcwd(), path), 'x', encoding='utf-8') as f: #if file not exists
            json.dump(data, f, indent = 2)
    except FileExistsError:
        if rewrite == 0:
            with open(os.path.join(os.getcwd(), path), 'a', encoding='utf-8') as f: #if file exists
                json.dump(data, f, indent = 2)
        else:
            with open(os.path.join(os.getcwd(), path), 'w', encoding='utf-8') as f: #if file exists
                json.dump(data, f, indent = 2)
    except ValueError:                               #if file is not json or json error
        print('Invalid json: ' + path)

In [25]:
def openJson(path):
    try:
        with open(os.path.join(os.getcwd(), path), 'r', encoding='utf-8') as f:        
            dataJSON = json.load(f)
            return dataJSON
    except FileNotFoundError:                        #if file not found
        print('File Not Found: ' + path)
        return -1
    except ValueError:                               #if file is not json
        print('Invalid json: ' + path)
        return -2

In [26]:
# Task: [{ "file path" : "path", 'hash file" : "hash" }, ........]

In [27]:
def fileToHash(fpath, hpath):
    hashsum_md5 = md5(fpath)   
    hashsum_sha256 = sha256(fpath) 
    data = openJson(hpath)
    if data == -1:
        writeToJson(hpath, [{"file_path" : fpath, "hash_file_md5" : hashsum_md5, "hash_file_sha256" : hashsum_sha256}], 1)
        print("Creation file!")
    elif data == -2:
        print("Something went wrong!")
    else:
        for i in range(len(data)):
            if(data[i].get('hash_file_md5') == hashsum_md5 or data[i].get('hash_file_sha256') == hashsum_sha256):
                print("Hash of file exists!")
                break
            else:
                if i == len(data) - 1:
                    data = openJson(hpath)
                    data.append({"file_path" : fpath, "hash_file_md5" : hashsum_md5, "hash_file_sha256" : hashsum_sha256})
                    writeToJson(hpath, data, 1)
                    print("Hash Added!")
                

In [28]:
df = pd.read_csv('Cleaned_Laptop_data.csv', index_col = 0)
df

,model,processor_brand,processor_name,processor_gnrtn,ram_gb,ram_type,ssd,hdd,os,os_bit,...,display_size,warranty,Touchscreen,msoffice,latest_price,old_price,discount,star_rating,ratings,reviews
brand,,,,,,,,,,,,,,,,,,,,,
Lenovo,A6-9225,AMD,A6-9225 Processor,10th,4 GB GB,DDR4,0 GB,1024 GB,Windows,64-bit,...,Missing,0,No,No,24990,32790,23,3.7,63,12
Lenovo,Ideapad,AMD,APU Dual,10th,4 GB GB,DDR4,0 GB,512 GB,Windows,64-bit,...,Missing,0,No,No,19590,21325,8,3.6,1894,256
Avita,PURA,AMD,APU Dual,10th,4 GB GB,DDR4,128 GB,0 GB,Windows,64-bit,...,Missing,0,No,No,19990,27990,28,3.7,1153,159
Avita,PURA,AMD,APU Dual,10th,4 GB GB,DDR4,128 GB,0 GB,Windows,64-bit,...,Missing,0,No,No,21490,27990,23,3.7,1153,159
Avita,PURA,AMD,APU Dual,10th,4 GB GB,DDR4,256 GB,0 GB,Windows,64-bit,...,Missing,0,No,No,24990,33490,25,3.7,1657,234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ASUS,ROG,AMD,Ryzen 9,Missing,4 GB GB,DDR4,1024 GB,0 GB,Windows,64-bit,...,16,1,No,No,144990,194990,25,0.0,0,0
ASUS,Ryzen,AMD,Ryzen 9,Missing,4 GB GB,DDR4,1024 GB,0 GB,Windows,64-bit,...,Missing,1,No,No,149990,0,0,0.0,0,0
ASUS,Ryzen,AMD,Ryzen 9,Missing,4 GB GB,DDR4,1024 GB,0 GB,Windows,64-bit,...,16,1,No,No,142990,0,0,0.0,0,0


In [29]:
df.to_parquet('Cleaned_Laptop_data.parquet')

In [30]:
md5('Cleaned_Laptop_data.parquet'), sha256('Cleaned_Laptop_data.parquet')

('b11afe2ed5dad81f8e6fd2412be1ded4',
 '7c0df1f9a7c94184ec1712e265b49caba9de8d3f7aa2143bcca9ca2da1fa5d79')

In [31]:
fileToHash('Cleaned_Laptop_data.parquet', 'HashData.json')

Hash of file exists!


In [32]:
df = pd.read_parquet('Cleaned_Laptop_data.parquet')

In [33]:
df.loc[df['processor_brand'] == 'Intel'].to_parquet('Laptop_Intel.parquet')

In [34]:
fileToHash('Laptop_Intel.parquet', 'HashData.json')

Hash Added!


In [35]:
data = openJson('HashData.json') #all hash
for i in range(len(data)):
    #if(data[i].get('hash_file') != hashsum):
    print(data[i].get('hash_file_md5'), "\t", data[i].get('hash_file_sha256'))

b11afe2ed5dad81f8e6fd2412be1ded4 	 7c0df1f9a7c94184ec1712e265b49caba9de8d3f7aa2143bcca9ca2da1fa5d79
2142d8d452d904f8d9bfecaa5725066e 	 eb7e2155deb7a1a847e5959c611872493e4fa33f73a39d295f8de57a9ffc051c


# Task 3

In [60]:
# (a-b)^2 - (b+c)*d + (b+c)(a-b)
a = 1
b = 2
c = 3
d = 4
default_args={
    "owner":"airflow",
    "start_date":datetime.today()-timedelta(days=1),
}

dag = DAG(dag_id="algebra", default_args=default_args)


def xMinY(x, y):
    return x - y

def xPlusY(x, y):
    return x + y

def xByY(x, y):
    return x * y

def xPow2(x):
    return x * x

t1 = PythonOperator (
    task_id = 'a_minus_b',
    python_callable = xMinY,
    op_kwargs = {"x" : a, "y" : b},
    dag = dag
)

t2 = PythonOperator (
    task_id = 'b_plus_c',
    python_callable = xPlusY,
    op_kwargs = {"x" : b, "y" : c},
    dag = dag    
)

t3 = PythonOperator (
    task_id = 'bc_by_ab',
    python_callable = xPlusY,
    op_kwargs = {"x" : b, "y" : c},
    dag = dag    
)

t1 >> t2

<Task(PythonOperator): b_plus_c>